In [3]:
import pandas as pd
import numpy as np

dados = pd.read_csv("CC_GENERAL.csv")
dados.drop(columns=['CUST_ID','TENURE'],axis=1,inplace=True)

In [4]:
faltando = dados.isna().sum()
faltando

BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
dtype: int64

In [7]:
dados.fillna(dados.median(), inplace=True)
faltando = dados.isna().sum()
faltando

BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
dtype: int64

In [8]:
from sklearn.preprocessing import Normalizer
values = Normalizer().fit_transform(dados.values)

In [9]:
from sklearn.cluster import KMeans
# n_clusters = numero de clusters
# n_init = roda 10x seguidas e retorna 10x o mesmo valor
# max_inter = numero maximo de interações
kmeans = KMeans(n_clusters=5, n_init=10, max_iter=300)
y_pred = kmeans.fit_predict(values)

Metricas de validação
Externas                    | Internas
precisamos ter os labels    | independente de labels


criterios de validação
Compactação: quão próximo estão os pontos em um mesmo cluster
Separação: quão bem separados estão os pontos em clusters diferentes

Coeficiente de Silhoutte
$$
\begin{equation}
    \begin{aligned}
       s = \frac{b - a}{max(a,b)} \\
       \\
        a &= \text{alpha, distância média entre o ponto e todos os outros pontos do mesmo cluster} \\
        b &= \text{beta , distância média entre o ponto e todos os outros pontos do cluster mais próximo} \\
    \end{aligned}
\end{equation}
$$

Distância Euclidiana
$$
\begin{equation}
    \begin{aligned}
       d(A , B) = \sqrt{(x_{B} - x_{A})^2 + (y_{B} - y_{A})^2} \\
    \end{aligned}
\end{equation}
$$